<a href="https://colab.research.google.com/github/maxtaq/GenomeBento/blob/master/SaveMyLabourRecordFireFox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新方針：要素クリック法でゆく。

# import

In [1]:
import sys, os
from datetime import time, date, datetime
import time

import numpy as np
import pandas as pd
import pandas.io.sql as psql

import re
import requests
import urllib.request as urllib2
import urllib3
from bs4 import BeautifulSoup

from statistics import mean, median,variance,stdev
from itertools import zip_longest

# pd.options.display.max_columns = None

In [2]:
# seleniumのPATHはインストールしないと設定されない　参考：https://qiita.com/ftoyoda/items/fe3e2fe9e962e01ac421
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

# seleniumのインストール
!pip install selenium

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:12 https://developer.downlo

In [ ]:
# 隠しシンプルログインページらしきもの
# 週報ログインタイムアウト後遷移して下記に飛ぶ
# https://cl.i-abs.co.jp/e-clocking/page//Xew0000.asp?OK.x=28&OK.y=11

## Chrome Driverでアクセス

In [31]:
# Selenium Chrome Webdriverの起動
# 参考：https://qiita.com/Azunyan1111/items/b161b998790b1db2ff7a

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import requests

# ブラウザのオプションを格納する変数をもらってきます。
options = Options()

# Headlessモードを有効にする（コメントアウトするとブラウザが実際に立ち上がります）
options.set_headless(True)
# options.setter(True)


# seleniumの起動オプション　参考：https://qiita.com/ftoyoda/items/fe3e2fe9e962e01ac421
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ブラウザを起動する
# driver = webdriver.Chrome(chrome_options=options)
driver = webdriver.Chrome(options=options)


# 暗黙的なWaitを設定する
# driver.implicitly_wait(5)

# ヘッダを下記のように指定
headers = {"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"}

# Selenium(Google Chrome Headlessモード）でアクセスする
driver.get("https://cl.i-abs.co.jp/e-clocking/login.asp")
time.sleep(2)

# HTMLを文字コードをShift-JISに変換してから取得します。
# html_of_top = driver.page_source.encode('Shift-JIS')
# display(html_of_top)


###################
# frame内に移動する
###################

# この節は上記コードの直後に実行しないと動作しない。
# driveが要素を格納するのか。
iframe = driver.find_element_by_css_selector('html > frameset > frame:nth-child(2)')
driver.switch_to.frame(iframe)

############
# hashを取得
############

s_frame = BeautifulSoup(driver.page_source, 'html5lib')

# Curl時とはRegExが異なるので。
p = re.compile("(?<=\"hidden\"\svalue=\").{0,9999}(?=\"\/>)")

# BF取得に時間がかかるので
time.sleep(1)

_hash = (p.findall(str(s_frame)))[2]
print(_hash)

############
# ログインボタンを押す
############

# 単純な方法として以下の方法を採用する。
# https://stackoverflow.com/questions/56621291/how-to-submit-form-data-with-chromedriver


# エレガントで、高度な方法はこれ
# https://stackoverflow.com/questions/24841632/how-to-get-contents-of-frames-automatically-if-browser-does-not-support-frames


input_ksc = driver.find_element_by_css_selector("#id_maintbl > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(2) > input")
input_number = driver.find_element_by_css_selector("#id_maintbl > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(4) > input")
input_password = driver.find_element_by_css_selector("#id_maintbl > tbody > tr:nth-child(2) > td > table > tbody > tr > td:nth-child(6) > input.ldata")

input_ksc.send_keys("ksc")
input_number.send_keys("346")
input_password.send_keys("mojyarak")

driver.find_element_by_css_selector("#id_maintbl > tbody > tr:nth-child(3) > td > a > img").click()


############
# ログイン後ページを取得
############

SourceOfInRestrictedArea = BeautifulSoup(driver.page_source, 'html5lib')
display(len(SourceOfInRestrictedArea))

# 自番号を取得。
p_number = re.compile('''(?<=strEmpLogCode \= ").{0,3}(?=\"\;)''')

# BF取得に時間がかかるので
time.sleep(2)

_myid = (p_number.findall(str(SourceOfInRestrictedArea)))
display(_myid)

############
# 「届出処理メニュー」ページを取得
############

driver.find_element_by_css_selector('''body > form > table:nth-child(1) > tbody > tr > td > table > tbody > tr > td.apply_td > input[type=image]
''').click()

SourceOfTodokedeSyoriMenue = BeautifulSoup(driver.page_source, 'html5lib')
display(len(SourceOfTodokedeSyoriMenue))


# 就業週報へのリンクを取得。
p_syugyo = re.compile('''(?<=a href='.).{0,999}(?='>勤務変更届出)''')
_syugyo = (p_number.findall(str(SourceOfTodokedeSyoriMenue)))
syugyo_url = f'https://cl.i-abs.co.jp/e-clocking/page{_syugyo}'
display(_syugyo)

# Selenium(Google Chrome Headlessモード）でアクセスする
driver.get(syugyo_url)
time.sleep(1)


############
# 「就業週報」ページを取得
############

# 取得期間を本日から一か月前とする

import datetime
today = datetime.date.today()
first = today.replace(day=1)
_lastMonth = first - datetime.timedelta(days=1)
lastMonth = _lastMonth.strftime("%Y/%m/%d") 
display(lastMonth)

# 取得期間を入力
input_period_from = driver.find_element_by_css_selector('''body > table:nth-child(4) > tbody > tr:nth-child(1) > td > table.tbl_mgn > tbody > tr > td:nth-child(3) > input''')
input_period_to = driver.find_element_by_css_selector('''body > table:nth-child(4) > tbody > tr:nth-child(1) > td > table.tbl_mgn > tbody > tr > td:nth-child(5) > input''')

input_period_from.send_keys(today)
input_period_to.send_keys(lastMonth)

#　無効化・隠されたボタンを押す（押せるか不明）
driver.find_element_by_css_selector('''body > table:nth-child(4) > tbody > tr:nth-child(1) > td > table.tbl_mgn > tbody > tr > td:nth-child(7) > input[type=button]''').click()

# .btnDef-Blue
# 

MyLabourRecord = BeautifulSoup(driver.page_source, 'html5lib')
print(MyLabourRecord)



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: DeprecationWarning: use setter for headless property instead of set_headless


fe1b0b4c795ac103a83b4cfe6e4a163f


1

['346']

1

['346']

'2021/04/30'

NoSuchElementException: ignored

In [24]:
from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

today = datetime.today()
one_month_ago = today - relativedelta(month=1)

display(one_month_ago)
print(
    datetime.strftime(today, '%Y/%m/%d')
    ,datetime.strftime(one_month_ago, '%Y/%m/%d')
    )

# dt_now = datetime.datetime.now()
# print(f'{dt_now.year}/{dt_now.month}/{dt_now.day}')

datetime.datetime(2021, 1, 31, 6, 17, 7, 901328)

2021/05/31 2021/01/31


In [28]:
import datetime
today = datetime.date.today()
# ex: 2021-05-01
first = today.replace(day=1)
_lastMonth = first - datetime.timedelta(days=1)
lastMonth = _lastMonth.strftime("%Y/%m/%d") 
display(lastMonth)


'2021/04/30'

In [4]:
SourceOfInRestrictedArea

<html lang="ja"><head>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="text/css" http-equiv="content-style-type"/>
<meta content="text/javascript" http-equiv="content-script-type"/>
<meta content="no-cache" http-equiv="pragma"/>
<meta content="no-cache" http-equiv="cache-control"/>
<title>e-clocking 打刻登録</title>
<meta content="none" name="robots"/>
<meta content="CyberXeed 就業・給与・人事 Web Service" name="description"/>
<meta content="ABS-ASE:Development Team" name="author"/>
<link href="./images/e-clocking.ico" rel="shortcut icon" type="image/vnd.microsoft.icon"/>
<link href="../css/XewCommonDesign.css?20140704_151200" media="all" rel="stylesheet"/>
<script src="../js/navigator.js?20140704_151200" type="text/javascript"></script>
<script src="../js/crossbrowser.js?20140704_151200" type="text/javascript"></script>

<script src="../js/cookie.js?20140704_151200" type="text/javascript"></script>

<script type="text/javascript">

function init()
{
	var strCom

In [ ]:

# ソースを取得
_rs = requests.Session()

# hashを取得
rsg_hash = _rs.get('https://cl.i-abs.co.jp/e-clocking/page/Xew0000.asp')
soup_rsg = BeautifulSoup(rsg_hash.text)
p = re.compile("(?<=\"hidden\"\svalue=\").{0,9999}(?=\"\/>)")
time.sleep(1)
_hash2 = (p.findall(str(soup_rsg)))
display("hash2", _hash2)


# driver.find_element_by_css_selector('.some-selector-in-iframe').click()

payload = {
    'PROCESS': '',
    'PassData': '',
    'DataSource': 'ksc',
    'LoginID': '346',
    'PassWord': 'mojyarak',
    'hash': '{_hash}'.format(_hash = _hash2)
}

# ログイン（postコマンド）
_rs.post("https://cl.i-abs.co.jp/e-clocking/page/Xew0000.asp", data=payload)

time.sleep(3)

# ログイン後改めて、ページを取得
rsg = _rs.get('https://cl.i-abs.co.jp/e-clocking/page/Xew0000.asp')
soup_rsg = BeautifulSoup(rsg.text)

display(str(soup_rsg))

_myid = (p.findall(str(soup_rsg)))
print(_myid)

In [ ]:
############
# 遷移後フレーム内要素を取得
############

# ログイン後改めてこのuriでアクセスしなおす必要ありそうだ。
driver.get("https://cl.i-abs.co.jp/e-clocking/login.asp")
# time.sleep(1)

# この節は上記コードの直後に実行しないと動作しない。
iframeInRestrictedArea = driver.find_element_by_css_selector('html > frameset > frame:nth-child(2)')
driver.switch_to.frame(iframeInRestrictedArea)

SourceOfInRestrictedArea = BeautifulSoup(driver.page_source, 'html5lib')
display(len(SourceOfInRestrictedArea))

# 自番号を取得。
p = re.compile('''(?<=strEmpLogCode \= ").{0,3}(?=\"\;)''')

# BF取得に時間がかかるので
time.sleep(2)

_myid = (p.findall(str(s_frame)))
print(_myid)


############
# 週報につながるボタンを押す
############


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: use driver.switch_to.frame instead
  """


bb7e82b7fed00377d683d8f5e9773fff


# 旧コード

In [ ]:
# # # # # # # # # # # # # # # # # #
# Topページからhashを取得 tag選択
# # # # # # # # # # # # # # # # # #

!curl https://cl.i-abs.co.jp/e-clocking/page/Xew0000.asp | iconv -f Shift-JIS -t UTF8 > /content/login.html
curled_html = open("/content/login.html", "r")
s_curl = curled_html.read()

# soup = BeautifulSoup(r.text)
p = re.compile("(?<=\"hash\"\s\svalue\=\").{0,9999}(?=\"\>)")
# print(s)

_hash = p.findall(s_curl)
# display(str(s))
#　他のやり方：myname = soup.find_all("span", class_="header_nickname")
print(_hash)

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 18729  100 18729    0     0  41712      0 --:--:-- --:--:-- --:--:-- 41620
['0bcf5fef3cf1bd0e28a169cea360184d']


## Selenium Install

In [ ]:
######

# Login ver2

######

# Selenium Chrome Webdriverの起動
# 参考：https://qiita.com/Azunyan1111/items/b161b998790b1db2ff7a

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

import requests

# ブラウザのオプションを格納する変数をもらってきます。
options = Options()

# Headlessモードを有効にする（コメントアウトするとブラウザが実際に立ち上がります）
options.set_headless(True)
# options.setter(True)


# seleniumの起動オプション　参考：https://qiita.com/ftoyoda/items/fe3e2fe9e962e01ac421
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

# ブラウザを起動する
# driver = webdriver.Chrome(chrome_options=options)
driver = webdriver.Chrome(options=options)


# 暗黙的なWaitを設定する
# driver.implicitly_wait(5)

# ヘッダを下記のように指定
headers = {"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"}

# Selenium(Google Chrome Headlessモード）でアクセスする
driver.get("https://cl.i-abs.co.jp/e-clocking/login.asp")
time.sleep(2)

# HTMLを文字コードをShift-JISに変換してから取得します。
# html_of_top = driver.page_source.encode('Shift-JIS')
# display(html_of_top)


###################
# frame内に移動する
###################

# この節は上記コードの直後に実行しないと動作しない。
# driveが要素を格納するのか。
iframe = driver.find_element_by_css_selector('html > frameset > frame:nth-child(2)')
driver.switch_to.frame(iframe)

############
# hashを取得
############

s_frame = BeautifulSoup(driver.page_source, 'html5lib')

# Curl時とはRegExが異なるので。
p = re.compile("(?<=\"hidden\"\svalue=\").{0,9999}(?=\"\/>)")

# BF取得に時間がかかるので
time.sleep(1)

_hash = (p.findall(str(s_frame)))[2]
print(_hash)

########
# login
########

from bs4 import BeautifulSoup
import requests

# type = hiddenも含めること。
payload = {
    'PROCESS': '',
    'PassData': '',
    'DataSource': 'ksc',
    'LoginID': '346',
    'PassWord': 'mojyarak',
    'hash': '{_hash}'.format(_hash = _hash)
}

s = requests.Session()

########
セッション内の_hashを取得
########




########
# ログイン（postコマンド）
########
s.post('https://cl.i-abs.co.jp/e-clocking/page/Xew0000.asp', data=payload)

#######

payload = {'uid' : 'username',
          'pwd' : 'password'}

with requests.Session() as session:
    r = session.post(r'http://192.168.2.114(cgi-bin/wwwugw.cgi', data=payload)

    print(r.json)


#######
# フレームの移動
#######

# iframe = driver.find_element_by_css_selector('html > frameset > frame:nth-child(2)')
# driver.switch_to.frame(iframe)

# r = s.get('https://cl.i-abs.co.jp/e-clocking/page/Xew0000.asp')
r = s.get('https://cl.i-abs.co.jp/e-clocking/login.asp')

# time.sleep(3)


soup = BeautifulSoup(r.text)
# time.sleep(3)
# display(soup)
# 自番号を取得。

p = re.compile('''(?<=strEmpLogCode \= ").{0,3}(?=\"\;)''')
_myid = p.findall(str(soup))
print(_myid)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: DeprecationWarning: use setter for headless property instead of set_headless


08d3e320375c6b4dfaa60bdaf4dcb9e2
[]


In [ ]:
import requests
payload = {'uid' : 'username',
          'pwd' : 'password'}

with requests.Session() as session:
    r = session.post(r'http://192.168.2.114(cgi-bin/wwwugw.cgi', data=payload)

    print(r.json)

## login

## 保存

In [ ]:
import requests
from selenium import webdriver

cookie_name = 'session_id'

# requests でログインしてセッションクッキーを作る
s = requests.session()
login_data = {
    'PROCESS': '',
    'PassData': '',
    'DataSource': 'ksc',
    'LoginID': '346',
    'PassWord': 'mojyarak',
    'hash': '{_hash}'.format(_hash = _hash),
}
res = s.post('https://cl.i-abs.co.jp/e-clocking/login.asp', data=login_data)
res.raise_for_status()

# セッションクッキーの値
cookie_value = s.cookies.get(cookie_name)

# Firefox起動
driver = webdriver.Firefox()
# 事前に、ターゲットドメインの適当なページを開いておく。
# これをしないと、add_cookie でエラーになる
driver.get('https://example.com/')
# requests で取得したセッションクッキーを Firefox に渡す
driver.add_cookie({
        'name': cookie_name,
        'value': cookie_value,
        'domain': 'example.com'})
# ログイン後のページを表示
driver.get('https://example.com/mypage/')

In [ ]:
＃＃＃＃＃＃＃＃＃＃＃
ひな型
＃＃＃＃＃＃＃＃＃＃＃

import requests
from selenium import webdriver

cookie_name = 'session_id'

# requests でログインしてセッションクッキーを作る
s = requests.session()
login_data = {
    'email': 'xxxx@xxxx.xx',
    'password': 'xxxxxxxxxx',
}
res = s.post('https://example.com/login/', data=login_data)
res.raise_for_status()

# セッションクッキーの値
cookie_value = s.cookies.get(cookie_name)

# Firefox起動
driver = webdriver.Firefox()
# 事前に、ターゲットドメインの適当なページを開いておく。
# これをしないと、add_cookie でエラーになる
driver.get('https://example.com/')
# requests で取得したセッションクッキーを Firefox に渡す
driver.add_cookie({
        'name': cookie_name,
        'value': cookie_value,
        'domain': 'example.com'})
# ログイン後のページを表示
driver.get('https://example.com/mypage/')

In [ ]:
!pip install selenium_requests
!pip install selenium-firefox

     |████████████████████████████████| 13.9MB 280kB/s 


In [ ]:
!pip install geckodriver-autoinstaller

In [ ]:
# Import any WebDriver class that you would usually import from
# selenium.webdriver from the seleniumrequests module
# from seleniumrequests import Firefox
from seleniumrequests import Firefox

# Simple usage with built-in WebDrivers:
webdriver = Firefox()
# webdriver = webdriver.Chrome(options=options)

response = webdriver.request('GET', 'https://www.google.com/')
print(response)


# More complex usage, using a WebDriver from another Selenium-related module:
from seleniumrequests.request import RequestMixin
from someothermodule import CustomWebDriver


class MyCustomWebDriver(RequestMixin, CustomWebDriver):
    pass


custom_webdriver = MyCustomWebDriver()
response = custom_webdriver.request('GET', 'https://www.google.com/')
print(response)

WebDriverException: ignored